# All libraries

In [2]:
from pandas import Series, DataFrame
import statsmodels.api as sm
import pandas as pd
import nltk
import operator
import re,string
from patsy import dmatrices
%pylab inline
from nltk.corpus import stopwords
from itertools import chain
from collections import Counter
import random
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
warnings.filterwarnings('ignore')
from nltk.util import ngrams
from sklearn.feature_extraction.text import TfidfVectorizer

Populating the interactive namespace from numpy and matplotlib


In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('google1.csv', encoding = "ISO-8859-1")

In [5]:
df.head()

,Title,Date Written,Rating,Current/Former,Job Title,Location,Recommendation?,Outlook,Main Text,Pros,Cons,Advice to management
0,"""Operations manager""","Oct 5, 2018",5.0,Current Employee,Operations Manager,"New York, NY",Recommends,Positive Outlook,I have been working at Google full-time (More ...,"Perks, food, spaces, culture, health care and ...","Flat org, hard to grow",NaN
1,"""Moving at the speed of light, burn out is ine...","Jun 21, 2013",4.0,Former Employee,Program Manager,"Mountain View, CA",Doesn't Recommend,Negative Outlook,Show More,"1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,NaN
2,"""Great balance between big-company security an...","May 10, 2014",5.0,Current Employee,Software Engineer III,"New York, NY",Recommends,Positive Outlook,Show More,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",NaN
3,"""The best place I've worked and also the most ...","Feb 8, 2015",5.0,Current Employee,Anonymous Employee,"Mountain View, CA",Recommends,Positive Outlook,Show More,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,NaN
4,"""Unique, one of a kind dream job""","Jul 19, 2018",5.0,Former Employee,Software Engineer,"Los Angeles, CA",Recommends,Neutral Outlook,Roll back the trend towards becoming a regular...,Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",NaN


In [6]:
df.shape

(7632, 12)

# Analysis of Pros and Cons

In [7]:
pros = df['Pros']
cons = df['Cons']

In [8]:
#clean punctuation and white spaces
pros_clean = pros.apply(lambda x:re.sub(r'[^\w\s]', ' ', x.lower()))
pros_clean = pros_clean.apply(lambda x:re.sub(r'(\\n+)', ' ', x))
pros_clean = pros_clean.apply(lambda x:re.sub(r'^b[\'\"\'"\"''\s]', '', x.lower()))
pros_clean = pros_clean.apply(lambda x:re.sub(r'\s+', ' ', x))

In [9]:
#tokenizing and removing stop words
stop = set(stopwords.words('english'))
punc = string.punctuation
pros_clean = pros_clean.apply(lambda x: [word for word in word_tokenize(x) if word not in stop])
pros_clean = pros_clean.apply(lambda x: [word for word in x if word not in punc])

In [10]:
#getting frequency distribution
all_pros = pros_clean.sum()

In [11]:
pros_dist = nltk.FreqDist(all_pros)
pros_dict = pd.DataFrame.from_dict(pros_dist,orient='index').reset_index()
pros_dict.columns = ['word','frequency']

In [12]:
pros_dict.head()

,word,frequency
0,perks,1206
1,food,1075
2,spaces,12
3,culture,885
4,health,55


In [13]:
type(pros_dict)

pandas.core.frame.DataFrame

In [14]:
pros_dict.sort_values(by='frequency', ascending=0)[:20]

,word,frequency
74,great,2998
6,work,2899
49,people,2185
107,good,1817
92,benefits,1219
0,perks,1206
33,company,1132
1,food,1075
31,google,1030
81,smart,909


# Bigrams

In [15]:
pro_bigrams = ngrams(all_pros, 2)
print(Counter(pro_bigrams).most_common(20))

[(('smart', 'people'), 475), (('free', 'food'), 461), (('place', 'work'), 340), (('work', 'life'), 332), (('life', 'balance'), 316), (('great', 'benefits'), 301), (('great', 'people'), 291), (('work', 'environment'), 276), (('people', 'great'), 261), (('great', 'perks'), 248), (('people', 'work'), 194), (('great', 'company'), 186), (('great', 'place'), 185), (('great', 'work'), 180), (('work', 'great'), 175), (('co', 'workers'), 163), (('great', 'culture'), 155), (('good', 'work'), 150), (('company', 'work'), 143), (('perks', 'great'), 127)]


In [16]:
pro_trigrams = ngrams(all_pros, 3)
print(Counter(pro_trigrams).most_common(20))

[(('work', 'life', 'balance'), 309), (('great', 'place', 'work'), 125), (('smart', 'people', 'great'), 85), (('great', 'people', 'great'), 63), (('good', 'work', 'life'), 62), (('great', 'work', 'environment'), 58), (('great', 'company', 'work'), 50), (('great', 'people', 'work'), 47), (('great', 'work', 'life'), 46), (('great', 'perks', 'great'), 46), (('best', 'place', 'work'), 43), (('free', 'food', 'great'), 41), (('good', 'place', 'work'), 41), (('life', 'balance', 'great'), 39), (('people', 'great', 'benefits'), 37), (('work', 'smart', 'people'), 35), (('smart', 'co', 'workers'), 35), (('work', 'environment', 'great'), 34), (('free', 'food', 'free'), 34), (('place', 'work', 'great'), 33)]


# Let's look at cons

In [17]:
cons_clean = cons.apply(lambda x:re.sub(r'[^\w\s]', ' ', x.lower()))
cons_clean = cons_clean.apply(lambda x:re.sub(r'(\\n+)', ' ', x))
cons_clean = cons_clean.apply(lambda x:re.sub(r'^b[\'\"\'"\"''\s]', '', x.lower()))
cons_clean = cons_clean.apply(lambda x:re.sub(r'\s+', ' ', x))

#tokenizing and removing stop words
stop = set(stopwords.words('english'))
punc = string.punctuation
cons_clean = cons_clean.apply(lambda x: [word for word in word_tokenize(x) if word not in stop])
cons_clean = cons_clean.apply(lambda x: [word for word in x if word not in punc])

#getting frequency distribution
all_cons = cons_clean.sum()

In [18]:
cons_dist = nltk.FreqDist(all_cons)
cons_dict = pd.DataFrame.from_dict(cons_dist,orient='index').reset_index()
cons_dict.columns = ['word','frequency']

In [19]:
cons_dict.sort_values(by='frequency', ascending=0)[:20]

,word,frequency
5,work,1583
157,company,1366
17,google,991
173,people,820
93,get,746
195,big,689
2,hard,633
24,management,558
171,many,543
174,much,512


In [20]:
con_bigrams = ngrams(all_cons, 2)
print(Counter(con_bigrams).most_common(20))

[(('work', 'life'), 274), (('life', 'balance'), 263), (('big', 'company'), 260), (('long', 'hours'), 139), (('large', 'company'), 134), (('hard', 'get'), 94), (('middle', 'management'), 93), (('mountain', 'view'), 85), (('feel', 'like'), 81), (('get', 'promoted'), 74), (('many', 'people'), 72), (('smart', 'people'), 68), (('working', 'google'), 64), (('difficult', 'get'), 56), (('hard', 'work'), 55), (('things', 'done'), 54), (('work', 'hard'), 53), (('get', 'things'), 49), (('red', 'tape'), 48), (('full', 'time'), 46)]


In [21]:
con_trigrams = ngrams(all_cons, 3)
print(Counter(con_trigrams).most_common(20))

[(('work', 'life', 'balance'), 261), (('get', 'things', 'done'), 46), (('google', 'big', 'company'), 25), (('many', 'smart', 'people'), 20), (('great', 'place', 'work'), 19), (('cons', 'working', 'google'), 19), (('nothing', 'nothing', 'nothing'), 17), (('like', 'big', 'company'), 17), (('hard', 'get', 'promoted'), 16), (('hard', 'get', 'things'), 16), (('work', 'long', 'hours'), 15), (('big', 'company', 'hard'), 13), (('easy', 'get', 'lost'), 13), (('long', 'work', 'hours'), 13), (('feel', 'like', 'small'), 12), (('good', 'work', 'life'), 12), (('slow', 'decision', 'making'), 11), (('get', 'anything', 'done'), 11), (('life', 'balance', 'difficult'), 11), (('big', 'company', 'problems'), 11)]


In [22]:
!pip install -U gensim

Requirement already up-to-date: gensim in c:\programdata\anaconda3\lib\site-packages (3.6.0)
Requirement not upgraded as not directly required: smart-open>=1.2.1 in c:\programdata\anaconda3\lib\site-packages (from gensim) (1.7.1)
Requirement not upgraded as not directly required: numpy>=1.11.3 in c:\programdata\anaconda3\lib\site-packages (from gensim) (1.14.3)
Requirement not upgraded as not directly required: scipy>=0.18.1 in c:\programdata\anaconda3\lib\site-packages (from gensim) (1.1.0)
Requirement not upgraded as not directly required: six>=1.5.0 in c:\programdata\anaconda3\lib\site-packages (from gensim) (1.11.0)
Requirement not upgraded as not directly required: boto>=2.32 in c:\programdata\anaconda3\lib\site-packages (from smart-open>=1.2.1->gensim) (2.48.0)
Requirement not upgraded as not directly required: boto3 in c:\programdata\anaconda3\lib\site-packages (from smart-open>=1.2.1->gensim) (1.9.21)
Requirement not upgraded as not directly required: bz2file in c:\programdata\

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [23]:
from gensim.models.tfidfmodel import TfidfModel
import gensim

# Let's combine both pros and cons into a column 

In [24]:
df['Pros_Cons'] = df['Pros'] + df['Cons']

# TFIDF

In [25]:
vect = TfidfVectorizer(min_df = 5).fit(df['Pros_Cons']) ## Using our custom Tokenizer function using Stemming

In [26]:
## Total number of unique vocab after removing not-so-useful-words
len(vect.get_feature_names())

2744

In [27]:
X_vectorized = vect.transform(df['Pros_Cons'])

In [28]:
X_vectorized.toarray

<bound method _cs_matrix.toarray of <7632x2744 sparse matrix of type '<class 'numpy.float64'>'
	with 176470 stored elements in Compressed Sparse Row format>>

In [29]:
X_vectorized.shape

(7632, 2744)

In [30]:
doc =0
feature_index = X_vectorized[doc,:].nonzero()[1]
feature_names = vect.get_feature_names()

In [31]:
feature_index

array([2698, 2467, 2249, 1793, 1698, 1404, 1114, 1099, 1072,  961,  564,
        340,  129])

In [32]:
tfidf_scores = zip(feature_index, [X_vectorized[doc, x] for x in feature_index])

In [33]:
di = {}
for w, s in [(feature_names[i],s) for (i, s) in tfidf_scores]:
    di[w] = s

In [34]:
#di = sorted(di, lambda x )
sorted_by_value = sorted(di.items(), key=lambda kv: kv[1], reverse=True)

In [35]:
di

{'work': 0.1232610070805007,
 'to': 0.10518537008554617,
 'spaces': 0.4839252883931002,
 'perks': 0.1923677026601747,
 'org': 0.3874682226001394,
 'life': 0.22158148300243125,
 'health': 0.3842343894640443,
 'hard': 0.23140797505806665,
 'grow': 0.33036879104158556,
 'food': 0.19871400478340195,
 'culture': 0.20369984560500268,
 'care': 0.31689873548618547,
 'and': 0.10765138519794966}

In [36]:
type(di)

dict

In [37]:
pd.DataFrame(list(di.items()), columns=['Word', 'TFIDF Score']).sort_values(by='TFIDF Score', ascending=False)

,Word,TFIDF Score
2,spaces,0.483925
4,org,0.387468
6,health,0.384234
8,grow,0.330369
11,care,0.316899
7,hard,0.231408
5,life,0.221581
10,culture,0.203700
9,food,0.198714
3,perks,0.192368


In [38]:
pnc = df['Pros_Cons']

In [44]:
core_values = 'Focus on the user and all else will follow. It’s best to do one thing really, really well. Fast is better than slow. Democracy on the web works. You don’t need to be at your desk to need an answer. You can make money without doing evil. There’s always more information out there. The need for information crosses all borders. You can be serious without a suit. Great just isn’t good enough.'

In [40]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]



In [45]:
cosine_sim(pros_clean.sum(), core_values)

AttributeError: 'list' object has no attribute 'lower'

In [41]:
cosine_sim('a little bird', 'a little bird')

0.9999999999999998